# Objective
The goal of this notebook is to guide you through the process of executing an optimization process in Mlos.

# Steps
1. Select build configuration.
1. Generate the secrets file.
1. Build the docker image.
1. Launch the docker container.
1. Launch Mlos.Agent.
1. Launch SmartCache.exe benchmark.


# Prerequisites

1. Build the Mlos project. The build configuration (~~Debug~~ vs. __Retail__) must match your choice below.
1. Have Docker installed and available.
1. This notebook is run with `{MLOS_ROOT}\source\Mlos.Notebooks` as a working directory (default if you open from ADS or VS Code).

In [1]:
import json
import os
import subprocess
import time

from utils import generate_random_password, build_docker_image, run_docker_container, stop_docker_container, remove_docker_container
mlos_root_directory = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Select build configuration

__ATTENTION__: Optimization of DEBUG builds is pointless. Select DEBUG __only__ if you are trying to debug these projects.

In [2]:
build_configuration = 'Release' # you can change it to 'Debug' here.

assert build_configuration in ('Release', 'Debug')

if build_configuration == 'Release':
    obj_dir = "obj"
else:
    obj_dir = "objd"

mlos_agent_server_exe_path = os.path.abspath(os.path.join(mlos_root_directory, "out/dotnet/source/Mlos.Agent.Server", obj_dir, "AnyCPU", "Mlos.Agent.Server.exe"))
smart_cache_exe_path = os.path.abspath(os.path.join(mlos_root_directory, "out/dotnet/source/Examples/SmartCache", obj_dir, "x64", "SmartCache.exe"))

if not (os.path.exists(mlos_agent_server_exe_path) and os.path.exists(smart_cache_exe_path)):
    print(f"Are you sure you have done a build in a {build_configuration} configuration?")

# Generate the secrets file

>Note: We are moving away from using SqlRPC and towards gRPC for communication between Mlos.Agent and the Optimizer Service. However, until that migration is complete, Mlos.Agent has to know how to connect to Sql Server.

The secrets file is a simple json file generated by the code cell below.

In [3]:
overwrite_secrets = False

secrets_file_path = os.path.abspath(os.path.join(mlos_root_directory, "source/Mlos.Python/Secrets/local_docker_connection_string.json"))

if os.path.exists(secrets_file_path) and not overwrite_secrets:
    # Secrets already exist. We need to grab the password for later use.
    #
    print(f"Secrets file {secrets_file_path} already exists.")
    
    with open(secrets_file_path, 'r') as in_file:
        secrets = json.load(in_file)
    sa_password = secrets['Password']
    
else:
    # We need to create the secrets file from the template.
    sa_password = generate_random_password()
    sample_secrets_file_path = os.path.abspath(os.path.join(mlos_root_directory, "source/Mlos.Python/Secrets/sample_docker_connection_string.json"))
    with open(sample_secrets_file_path, 'r') as in_file:
        secrets_dict = json.load(in_file)
    
    secrets_dict['Password'] = sa_password
    with open(secrets_file_path, 'w') as out_file:
        json.dump(secrets_dict, out_file, indent=2)
    
    print(f"Wrote a new secrets file to {secrets_file_path}.")

Secrets file c:\Users\bpkroth\src\MLOS\MLOS.msdata.2\source\Mlos.Python\Secrets\local_docker_connection_string.json already exists.


# Build Docker image

The optimizer lives inside the Docker. Let's build the image.

In [4]:
build_docker_image(sa_password)


Step 1/15 : FROM microsoft/mssql-server-linux:latest
 ---> 314918ddaedf
Step 2/15 : RUN apt-get -y update &&     apt-get install -y software-properties-common &&     add-apt-repository -y ppa:deadsnakes/ppa &&     apt-get -y update &&     apt-get install -y python3.7 python3-pip &&     apt-get install -y build-essential libssl-dev libffi-dev python3-dev python3.7-dev unixodbc-dev &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c96a7f675e7e
Step 3/15 : RUN python3.7 -m pip install pip
 ---> Using cache
 ---> feb04a146619
Step 4/15 : RUN python3.7 -m pip install --upgrade pip
 ---> Using cache
 ---> 199aa4fecd21
Step 5/15 : RUN mkdir -p /usr/src/Mlos.Python
 ---> Using cache
 ---> 75325321633c
Step 6/15 : WORKDIR /usr/src/
 ---> Using cache
 ---> c28de51505c5
Step 7/15 : COPY ./requirements.txt ./Mlos.Python/requirements.txt
 ---> Using cache
 ---> 9e4576c57340
Step 8/15 : RUN python3.7 -m pip install -r /usr/src/Mlos.Python/requirements.txt
 ---> Using cache
 ---> dd3f294d79

# Run Docker container

In [5]:
run_docker_container()

docker run --detach -p1433:1433 --name MlosOptimizerService mssql-server-linux-with-mlos-python
bb2ea175d37a4428e885e17bae73e90c784514b779aad27fc0a4b850ea1a3f47


# Launch Mlos.Agent.Server

We need to launch Mlos.Agent.Server and let it communicate with the Optimizer.

In [6]:
print(mlos_agent_server_exe_path, f'"{secrets_file_path}"')
mlos_agent_server_process = subprocess.Popen([mlos_agent_server_exe_path, secrets_file_path], creationflags=subprocess.CREATE_NEW_CONSOLE)
time.sleep(15) # TODO: wait for Mlos.Agent.Server to signal readiness instead of blindly waiting.

c:\Users\bpkroth\src\MLOS\MLOS.msdata.2\out\dotnet\source\Mlos.Agent.Server\obj\AnyCPU\Mlos.Agent.Server.exe "c:\Users\bpkroth\src\MLOS\MLOS.msdata.2\source\Mlos.Python\Secrets\local_docker_connection_string.json"


# Launch SmartCache microbenchmark

In [7]:
print(smart_cache_exe_path)
smart_cache_process = subprocess.Popen([smart_cache_exe_path], creationflags=subprocess.CREATE_NEW_CONSOLE)

# wait for the process to exit
#
while smart_cache_process.poll() is None:
    time.sleep(1)
print("SmartCache.exe exited.")

c:\Users\bpkroth\src\MLOS\MLOS.msdata.2\out\dotnet\source\Examples\SmartCache\obj\x64\SmartCache.exe
SmartCache.exe exited.


# Clean up Docker

In [8]:
stop_docker_container()
remove_docker_container()

docker stop MlosOptimizerService
MlosOptimizerService
docker container rm MlosOptimizerService
MlosOptimizerService
